In [5]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col,sum


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

In [2]:
Sales = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Sales") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [3]:
Sales.show()

+----------+-------+--------+
| sale_date|  fruit|sold_num|
+----------+-------+--------+
|2020-05-01| apples|      10|
|2020-05-01|oranges|       8|
|2020-05-02| apples|      15|
|2020-05-02|oranges|      15|
|2020-05-03| apples|      20|
|2020-05-03|oranges|       0|
|2020-05-04| apples|      15|
|2020-05-04|oranges|      16|
+----------+-------+--------+



In [11]:
apple_sale_df = Sales\
    .filter(col("fruit")=='apples')\
    .groupBy(col("sale_date"))\
    .agg(sum("sold_num").alias("apple_sale"))

orange_sale_df = Sales\
    .filter(col("fruit")=='oranges')\
    .groupBy(col("sale_date"))\
    .agg(sum("sold_num").alias("orange_sale"))


apple_sale_df = apple_sale_df.withColumnRenamed("sale_date", "sale_dt")
apple_sale_df\
    .join(orange_sale_df, on=apple_sale_df.sale_dt == orange_sale_df.sale_date,how="inner")\
    .withColumn("diff",col("apple_sale")-col("orange_sale"))\
    .select("sale_dt","diff")\
    .show()

+----------+----+
|   sale_dt|diff|
+----------+----+
|2020-05-04|  -1|
|2020-05-02|   0|
|2020-05-03|  20|
|2020-05-01|   2|
+----------+----+

